In [1]:
import numpy as np
import pandas as pd
import re
import scipy
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## Загрузка данных

Считаем данные о квартирах с десяти страниц сайта (по 100шт на странице)

In [2]:
df_1 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF%20-%20%CF%F0%EE%E4%E0%EC%20%EA%E2%E0%F0%F2%E8%F0%F3%20%E2%20%E3.%20%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5")[16]
df_2 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=2")[16]
df_3 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=3")[16]
df_4 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=4")[16]
df_5 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=5")[16]
df_6 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=6")[16]
df_7 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=7")[16]
df_8 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=8")[16]
df_9 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=9")[16]
df_10 = pd.read_html("http://www.citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=10")[16]

In [3]:
print(df_10)

              0           1                                    2   \
0   просмотрфото        Дата                         Тип квартиры   
1   просмотрфото        Дата                         Тип квартиры   
2            NaN  25.1112:02               Двухкомнатная гребенка   
3            NaN  28.1208:02                        Трехкомнатная   
4            NaN         NaN                                  NaN   
5            NaN  29.1210:31                       Многокомнатная   
6            NaN  05.0520:00                        Однокомнатная   
7            NaN  27.1014:36                Трехкомнатная смежная   
8            NaN  13.1209:45             Однокомнатная улучшенная   
9            NaN  01.1219:53      Двухкомнатная старой планировки   
10           NaN  24.1119:42             Двухкомнатная улучшенная   
11           NaN  26.0215:49                        Однокомнатная   
12           NaN  01.1115:02     Двухкомнатная сталинский вариант   
13           NaN         NaN      

Из имеющихся данных, для построения модели прогнозирования стоимости квартиры, нам будут малополезны следующие столбцы:
* фото
* адрес (можно принебречь в пользу района)
* примечание
* цена по ипотеке
* телефон
* e-mail

In [4]:
columns_index = [1,2,3,5,6,7,8,9,10,13]
columns_name = ['date', 'type', 'area', 'floor', 'total_space', 
                'living_space', 'kitchen', 'description', 'price', 'agency']

flats_df = pd.concat([df_1.loc[2:, columns_index], df_2.loc[2:, columns_index], df_3.loc[2:, columns_index],
                     df_4.loc[2:, columns_index], df_5.loc[2:, columns_index], df_6.loc[2:, columns_index],
                     df_7.loc[2:, columns_index], df_8.loc[2:, columns_index], df_9.loc[2:, columns_index],
                     df_10.loc[2:, columns_index]], ignore_index=True)
flats_df.columns = columns_name

In [5]:
flats_df = flats_df.dropna(how='all').reset_index(drop=True)
flats_df

,date,type,area,floor,total_space,living_space,kitchen,description,price,agency
0,27.1212:39,Однокомнатная,Ленинский,1/3,37.00,18.00,7.00,Заключительный дом в п. Зелёная Долина ! Субси...,1650,Версаль
1,27.1212:44,Многокомнатная индивидуальный проект,Орджоникидзевский,16/16,96.00,70.00,15.00,"Продаётся квартира в новом современном доме, п...",4500,Версаль
2,27.1212:39,Трехкомнатная,Ленинский,2/3,70.00,60.00,8.00,Заключительный дом в п. Зелёная Долина ! Субси...,2699,Версаль
3,27.1212:39,Однокомнатная улучшенная,Орджоникидзевский,2/10,30.00,17.00,7.00,"Тёплая , светлая квартира , никто не проживает...",1670,Версаль
4,24.1213:26,Однокомнатная нестандартная,Орджоникидзевский,13/14,46.00,17.00,18.00,"Продается квартира в теплом, блочном доме, в п...",2600,NaN
...,...,...,...,...,...,...,...,...,...,...
918,27.1212:39,Трехкомнатная,Ленинский,3/4,117.00,60.00,20.00,"Квартира свободной планировки, в поселке Крыло...",4350,Версаль
919,10.1114:55,Трехкомнатная индивидуальный проект,Орджоникидзевский,13/14,79.00,48.00,14.00,"Продам 3-комнатную квартиру 13/14, 79/48/14, к...",4000,NaN
920,23.1018:24,Трехкомнатная свердловский вариант,Орджоникидзевский,1/9,62.00,50.00,9.00,"ПPОДAETСЯ OTЛИЧНАЯ 3х комнaтная квaртирa, ГОTO...",2790,NaN
921,18.0810:05,Трехкомнатная,Орджоникидзевский,3/3,103.20,75.00,15.00,Продам 4х комнатную квартиру на левом берегу! ...,2100,NaN


## Предобрабтка данных

In [6]:
flats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          923 non-null    object
 1   type          916 non-null    object
 2   area          863 non-null    object
 3   floor         923 non-null    object
 4   total_space   923 non-null    object
 5   living_space  923 non-null    object
 6   kitchen       923 non-null    object
 7   description   915 non-null    object
 8   price         923 non-null    object
 9   agency        652 non-null    object
dtypes: object(10)
memory usage: 72.2+ KB


### Date

In [7]:
#flats_df['date'] = pd.to_datetime('2021.' + flats_df['date'], format="%Y.%d.%m%H:%M")
flats_df['day'] = flats_df['date'].apply(lambda x: int(x[:2]))
flats_df['month'] = flats_df['date'].apply(lambda x: int(x[3:5]))

In [8]:
flats_df.head()

,date,type,area,floor,total_space,living_space,kitchen,description,price,agency,day,month
0,27.1212:39,Однокомнатная,Ленинский,1/3,37.00,18.00,7.00,Заключительный дом в п. Зелёная Долина ! Субси...,1650,Версаль,27,12
1,27.1212:44,Многокомнатная индивидуальный проект,Орджоникидзевский,16/16,96.00,70.00,15.00,"Продаётся квартира в новом современном доме, п...",4500,Версаль,27,12
2,27.1212:39,Трехкомнатная,Ленинский,2/3,70.00,60.00,8.00,Заключительный дом в п. Зелёная Долина ! Субси...,2699,Версаль,27,12
3,27.1212:39,Однокомнатная улучшенная,Орджоникидзевский,2/10,30.00,17.00,7.00,"Тёплая , светлая квартира , никто не проживает...",1670,Версаль,27,12
4,24.1213:26,Однокомнатная нестандартная,Орджоникидзевский,13/14,46.00,17.00,18.00,"Продается квартира в теплом, блочном доме, в п...",2600,NaN,24,12


### Type

In [9]:
# Т.к. в данном признаке присутствует всего семь пропущенных значений, удалим эти строчки
flats_df.dropna(subset=['type'], inplace=True)

In [10]:
flats_df['type'].value_counts()

Двухкомнатная                  219
Однокомнатная                  171
Трехкомнатная                  160
Двухкомнатная улучшенная        28
Трехкомнатная брежневка         25
                              ... 
Однокомнатная раздельная         1
Четырехкомнатная раздельная      1
Трехкомнатная стандартная        1
Трехкомнатная 121 серия          1
Четырехкомнатная смежная         1
Name: type, Length: 66, dtype: int64

На основании типа квартиры создадим признак, отражающий количество комнат в квартире

In [11]:
def rooms_count(row):
    rooms_list = list()
    if 'Однокомнатная' in row['type']:
        return 1
    elif 'Двухкомнатная' in row['type']:
        return 2
    elif 'Трехкомнатная' in row['type']:
        return 3
    elif 'Четырехкомнатная' in row['type']:
        return 4
    elif 'Многокомнатная' in row['type']:
        return 5

In [12]:
flats_df['rooms_count'] = flats_df.apply(rooms_count, axis=1)

In [13]:
flats_df.head()

,date,type,area,floor,total_space,living_space,kitchen,description,price,agency,day,month,rooms_count
0,27.1212:39,Однокомнатная,Ленинский,1/3,37.00,18.00,7.00,Заключительный дом в п. Зелёная Долина ! Субси...,1650,Версаль,27,12,1
1,27.1212:44,Многокомнатная индивидуальный проект,Орджоникидзевский,16/16,96.00,70.00,15.00,"Продаётся квартира в новом современном доме, п...",4500,Версаль,27,12,5
2,27.1212:39,Трехкомнатная,Ленинский,2/3,70.00,60.00,8.00,Заключительный дом в п. Зелёная Долина ! Субси...,2699,Версаль,27,12,3
3,27.1212:39,Однокомнатная улучшенная,Орджоникидзевский,2/10,30.00,17.00,7.00,"Тёплая , светлая квартира , никто не проживает...",1670,Версаль,27,12,1
4,24.1213:26,Однокомнатная нестандартная,Орджоникидзевский,13/14,46.00,17.00,18.00,"Продается квартира в теплом, блочном доме, в п...",2600,NaN,24,12,1


### Категориальные признаки

In [14]:
feats = ['type', 'area', 'agency']

In [15]:
flats_df[feats][:5].fillna('-').T.to_dict()

{0: {'type': 'Однокомнатная', 'area': 'Ленинский', 'agency': 'Версаль'},
 1: {'type': 'Многокомнатная индивидуальный проект',
  'area': 'Орджоникидзевский',
  'agency': 'Версаль'},
 2: {'type': 'Трехкомнатная', 'area': 'Ленинский', 'agency': 'Версаль'},
 3: {'type': 'Однокомнатная улучшенная',
  'area': 'Орджоникидзевский',
  'agency': 'Версаль'},
 4: {'type': 'Однокомнатная нестандартная',
  'area': 'Орджоникидзевский',
  'agency': '-'}}

In [16]:
vectorizer_feats = DictVectorizer()

flats_df_feats = vectorizer_feats.fit_transform(flats_df[feats].fillna('-').T.to_dict().values())

In [17]:
flats_df_feats

<916x134 sparse matrix of type '<class 'numpy.float64'>'
	with 2748 stored elements in Compressed Sparse Row format>

### Floor

Создадим признак, отражающий в процентном отношении номер этажа от общего числа этажей в доме, а так же выделим отдельно номер этажа, на котором располагается квартира

In [18]:
floor_pattern = re.compile('(\d+)[/]')
amount_floor_pattern = re.compile('[/](\d+)')

In [19]:
flats_df['floor_amount'] = flats_df.floor.apply(lambda x: int(amount_floor_pattern.findall(x)[0]))

In [20]:
flats_df['floor'] = flats_df.floor.apply(lambda x: int(floor_pattern.findall(x)[0]))

In [21]:
flats_df['floor_perc'] = flats_df['floor']/ flats_df['floor_amount']

In [22]:
flats_df.head()

,date,type,area,floor,total_space,living_space,kitchen,description,price,agency,day,month,rooms_count,floor_amount,floor_perc
0,27.1212:39,Однокомнатная,Ленинский,1,37.00,18.00,7.00,Заключительный дом в п. Зелёная Долина ! Субси...,1650,Версаль,27,12,1,3,0.333333
1,27.1212:44,Многокомнатная индивидуальный проект,Орджоникидзевский,16,96.00,70.00,15.00,"Продаётся квартира в новом современном доме, п...",4500,Версаль,27,12,5,16,1.000000
2,27.1212:39,Трехкомнатная,Ленинский,2,70.00,60.00,8.00,Заключительный дом в п. Зелёная Долина ! Субси...,2699,Версаль,27,12,3,3,0.666667
3,27.1212:39,Однокомнатная улучшенная,Орджоникидзевский,2,30.00,17.00,7.00,"Тёплая , светлая квартира , никто не проживает...",1670,Версаль,27,12,1,10,0.200000
4,24.1213:26,Однокомнатная нестандартная,Орджоникидзевский,13,46.00,17.00,18.00,"Продается квартира в теплом, блочном доме, в п...",2600,NaN,24,12,1,14,0.928571


### Площади

In [23]:
flats_df['total_space'] = flats_df['total_space'].apply(lambda x: float(x))
flats_df['living_space'] = flats_df['living_space'].apply(lambda x: float(x))
flats_df['kitchen'] = flats_df['kitchen'].apply(lambda x: float(x))

In [24]:
flats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 916 entries, 0 to 922
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          916 non-null    object 
 1   type          916 non-null    object 
 2   area          859 non-null    object 
 3   floor         916 non-null    int64  
 4   total_space   916 non-null    float64
 5   living_space  916 non-null    float64
 6   kitchen       916 non-null    float64
 7   description   908 non-null    object 
 8   price         916 non-null    object 
 9   agency        645 non-null    object 
 10  day           916 non-null    int64  
 11  month         916 non-null    int64  
 12  rooms_count   916 non-null    int64  
 13  floor_amount  916 non-null    int64  
 14  floor_perc    916 non-null    float64
dtypes: float64(4), int64(5), object(6)
memory usage: 146.8+ KB


### Description

In [25]:
vectorizer_description = TfidfVectorizer(min_df=3, max_df=0.3, ngram_range=(1, 3))

flats_df_description = vectorizer_description.fit_transform(flats_df['description'].fillna('-'))

In [26]:
vectorizer_description.vocabulary_

{'дом': 2205,
 'зелёная': 2886,
 'долина': 2192,
 'ипотека': 3137,
 'от': 6070,
 'ниже': 5428,
 'земельным': 2893,
 'участком': 10217,
 'собственная': 8963,
 'территория': 9631,
 'индивидуальный': 3059,
 'газовый': 1528,
 'котёл': 4189,
 'отдельный': 6188,
 'вход': 1332,
 'возможен': 1093,
 'вариант': 763,
 'ремонтом': 8445,
 'помогу': 7244,
 'оформлением': 6313,
 'ипотеки': 3152,
 'звоните': 2827,
 'расскажу': 8325,
 'покажу': 7054,
 'варианты': 779,
 'квартир': 3475,
 'зелёная долина': 2887,
 'квартира земельным': 3505,
 'земельным участком': 2894,
 'индивидуальный газовый': 3060,
 'газовый котёл': 1532,
 'отдельный вход': 6189,
 'возможен вариант': 1094,
 'помогу оформлением': 7248,
 'оформлением ипотеки': 6314,
 'ипотеки звоните': 3153,
 'звоните расскажу': 2855,
 'расскажу покажу': 8328,
 'все варианты': 1173,
 'квартира земельным участком': 3506,
 'помогу оформлением ипотеки': 7249,
 'продаётся': 7869,
 'новом': 5488,
 'современном': 9056,
 'доме': 2272,
 'комфорта': 4076,
 'стро

In [27]:
flats_df_description

<916x11049 sparse matrix of type '<class 'numpy.float64'>'
	with 107733 stored elements in Compressed Sparse Row format>

### Price

In [28]:
flats_df['price'] = flats_df['price'].apply(lambda x: float(x))

## Создание модели

Сформируем общий датасет, на основании созданных признаков, для обучения модели

In [29]:
df_columns = ['day', 'month', 'floor', 'floor_perc', 'rooms_count', 'total_space', 'living_space', 'kitchen']

In [30]:
X = scipy.sparse.hstack([np.matrix(flats_df[df_columns]), flats_df_feats, flats_df_description])
y = np.array([flats_df['price']]).T

Разделим данные на train и test

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Используем модель Ridge-регрессии

In [32]:
model = RidgeCV(alphas=(0.1, 1.0, 10.0))
model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [33]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print('Ошибка на трейне: {} тыс.руб.'.format(round(mean_squared_error(y_train, train_preds))))
print('Ошибка на тесте: {} тыс.руб.'.format(round(mean_squared_error(y_test, test_preds))))

Ошибка на трейне: 95981 тыс.руб.
Ошибка на тесте: 295264 тыс.руб.


In [35]:
X_new = flats_df[df_columns]
y_new = flats_df['price']
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.3)


model_2 = RidgeCV(alphas=(0.1, 1.0, 10.0))
model_2.fit(X_train_new, y_train_new)

train_preds_new = model_2.predict(X_train_new)
test_preds_new = model_2.predict(X_test_new)

print('Ошибка на трейне: {} тыс.руб.'.format(round(mean_squared_error(y_train_new, train_preds_new))))
print('Ошибка на тесте: {} тыс.руб.'.format(round(mean_squared_error(y_test_new, test_preds_new))))

Ошибка на трейне: 446472 тыс.руб.
Ошибка на тесте: 507290 тыс.руб.
